In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from typing import TypedDict, Annotated, Literal
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.types import Command, Send
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode
from langchain_core.tools import tool, InjectedToolCallId
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, SystemMessage
import operator
import json

In [3]:
class State(TypedDict):
    messages: Annotated[list, operator.add]
    pending_tool_calls: list[dict]
    tool_results: Annotated[list[dict], operator.add]
    needs_human_review: bool
    final_answer: str

## Tool calls with `InjectedToolCallId`

In [ ]:
import yfinance as yf
from datetime import datetime
import logging

logging.basicConfig(logging.INFO)
logger = logging.getLogger(__name__)

@tool
def search_for_stock_price(
    ticker: str,
    tool_call_id: Annotated[str, InjectedToolCallId],
    start_date: Annotated[str | None, "Date time in YYYY-MM-DD format"] = None,
    end_date: str | None = None,
) -> Command:
    """Searches Yahoo Finance for stock information

    Args:
        ticker (str): Ticker of interest. For example "META"
        tool_call_id (Annotated[str, InjectedToolCallId]): _description_

    Returns:
        Command: A node of the parent graph to route to
    """
    date_format = "%Y-%m-%d"
    try:
        stock_ticker = yf.Ticker(ticker)
        if start_date and end_date:
            try:
                start_date_formatted = datetime.strptime(start_date, date_format)
                end_date_formatted = datetime.strptime(end_date, date_format)
                data = stock_ticker.history(start=start_date_formatted, end=end_date_formatted).to_json()
            except Exception as e:
                logger.error(f"Faced the error: {e}. Getting data from max period")
                data = stock_ticker.history(period="max").to_json()
        data = stock_ticker.history(period="max").to_json()
        
        tool_msg = ToolMessage(
            content=data,
            tool_call_id=tool_call_id,
            name="search_for_stock_prices"
        )
        return Command(
            update={
                "messages": [tool_msg],
                "tool_results":[{
                    'tool': 'search_for_stock_prices',
                    'status':'success',
                    'results': data
                }]
            },
            goto="analyze_results"
        )
        
    except Exception as e:
        msg = f"Unable to find the info from {ticker} due to {e}. It is likely that the ticker is incorrect"
        logger.error(msg)
        tool_msg = ToolMessage(
            content=msg,
            tool_call_id=tool_call_id,
            name="search_for_stock_prices"
        )
        return Command(
            update={'messages': [tool_msg]},
            goto="handle_no_results"
        )
    

In [11]:
stock_ticker = yf.Ticker("META")
data = stock_ticker.history(period="max")

In [13]:
data.to_json()

'{"Open":{"1337313600000":41.7923603777,"1337572800000":36.30618151,"1337659200000":32.4102025272,"1337745600000":31.1777968486,"1337832000000":32.748117403,"1337918400000":32.6984269408,"1338264000000":31.2871238652,"1338350400000":28.5241560393,"1338436800000":28.3750760657,"1338523200000":28.7129909645,"1338782400000":27.0333486862,"1338868800000":26.5364108997,"1338955200000":25.9102671346,"1339041600000":26.8345728442,"1339128000000":26.3873286475,"1339387200000":27.0134695247,"1339473600000":27.311632193,"1339560000000":27.4905304438,"1339646400000":27.4805886781,"1339732800000":28.3353221776,"1339992000000":29.7764373312,"1340078400000":31.3467589002,"1340164800000":31.724428314,"1340251200000":31.4759599454,"1340337600000":32.2114255144,"1340596800000":32.6586688442,"1340683200000":32.4897099086,"1340769600000":32.26111827,"1340856000000":31.7641823556,"1340942400000":31.7244280414,"1341201600000":31.0585348519,"1341288000000":30.7206164605,"1341460800000":31.128106754,"1341547